# Notebook 21: Estimating Missing Data
***

In this notebook, we will dive deeper into the problem of estimating missing ratings/data. Specifically, we will conduct some **preprocessing** on our data matrix, which can account for differences in how different users rate different items. Some users tend to be kind raters, and some will be stingier with those 5-star reviews. Normalizing our utility matrix before performing the UV decomposition on it will account for these differences.

We'll need numpy for this notebook, so let's load it.

In [2]:
import numpy as np

<br>

### Exercise 1: Preprocessing the data matrix

In class we performed a few iterations to find the UV decomposition of the following matrix, where the rows correspond to different users, and the columns correspond to different items. The elements of the matrix are the users' ratings for each item. There two unknown values:
* User 3's rating for Item 2, and
* User 6's rating for Item 5.

In [3]:
M = np.array([[5,2,4,4,3],
              [3,1,2,4,1],
              [2,np.nan,3,1,4],
              [2,5,4,3,5],
              [2,5,4,3,5],
              [4,4,5,4,np.nan]])

We suggested in class that **preprocessing** the data matrix would lead to better results, by accounting for differences in the ways different users tend to rate items, and differences in item ratings. Some of the possible methods for preprocessing are:
* Subtract from each non-blank element $m_{ij}$ the average rating of user $i$
* Subtract from each non-blank element in column $j$ the average rating of item $j$
* Do both of these, in either order
* From element $m_{ij}$ subtract $\frac{1}{2} \times$ (the average of user $i$ + the average of item $j$)

Let's shoot for the stars and subtract from each non-missing element $m_{ij}$ the average rating of user $i$, then subtract from that intermediate matrix the average rating of item $j$.

In [4]:
# initialize
user_means = np.zeros(M.shape[0])
item_means = np.zeros(M.shape[1])

# TODO -- compute the mean rating for each user (not including blanks)
user_means = np.zeros(M.shape[0])  # <-- modify this!

# normalize M first by subtracting from each non-blank element that user's mean rating
M_norm = M.copy()
for u in range(len(user_means)):
    M_norm[u,:] -= user_means[u]

# TODO -- compute the mean rating for each user (not including blanks)
item_means = np.zeros(M.shape[1])  # <-- modify this!

# normalize M once more by subtracting from each non-blank element that item's mean rating
for i in range(len(item_means)):
    M_norm[:,i] -= item_means[i]

Note that whatever we subtract off from each element of the matrix during preprocessing, we need to *add that back in* when estimating the missing values after our UV decomposition. So **go back** to the code cell above and **add in** a matrix that is of the same size as $M$, whose elements are the total amount that we subtracted off from each element of $M$ during the preprocessing normalization.

In [82]:
# SOLUTION:

# initialize
user_means = np.zeros(M.shape[0])
item_means = np.zeros(M.shape[1])

# added for un-normalizing to make rating estimates:
M_sub = np.zeros(M.shape)

# TODO -- compute the mean rating for each user (not including blanks)
user_means = np.nanmean(M, axis=1)

# normalize M first by subtracting from each non-blank element that user's mean rating
M_norm = M.copy()
for u in range(len(user_means)):
    M_norm[u,:] -= user_means[u]
    M_sub[u,:] -= user_means[u]

# TODO -- compute the mean rating for each user (not including blanks)
item_means = np.nanmean(M_norm, axis=0)

# normalize M once more by subtracting from each non-blank element that item's mean rating
for i in range(len(item_means)):
    M_norm[:,i] -= item_means[i]
    M_sub[:,i] -= item_means[i]
    
print(M)
print(M_norm-M_sub)

[[ 5.  2.  4.  4.  3.]
 [ 3.  1.  2.  4.  1.]
 [ 2. nan  3.  1.  4.]
 [ 2.  5.  4.  3.  5.]
 [ 2.  5.  4.  3.  5.]
 [ 4.  4.  5.  4. nan]]
[[ 5.  2.  4.  4.  3.]
 [ 3.  1.  2.  4.  1.]
 [ 2. nan  3.  1.  4.]
 [ 2.  5.  4.  3.  5.]
 [ 2.  5.  4.  3.  5.]
 [ 4.  4.  5.  4. nan]]


So we currently have $M=M_{norm}-M_{sub}$, and we'll use $M_{norm}$ for our UV fitting.

**Reflect:** Why did we not compute the user and item rating means at the same time? Why did we have to normalize by the user ratings first, *then* compute the mean item ratings?

<br>

### Exercise 2: U and V!

Let's find the UV decomposition of M (`M_norm`) using 2 dimensional vectors.  Per the slides, we need to alternatingly compute:


$$x=u_{rs}=\frac{\sum_j v_{sj} (m_{rj} - \sum_{k \ne s} u_{rk}v_{kj} )}{\sum_j v_{sj}^2}$$

$$y=v_{rs}=\frac{\sum_i u_{ir} (m_{is} - \sum_{k \ne r} u_{ik}v_{ks} )}{\sum_i u_{ir}^2}$$

for $x$ in the $U$ matrix and $y$ in the $V$ matrix.

Let's start with a couple of "easy" updates, and initialize U and V as all-ones, then update `u[0,0]` and `v[0,0]`.


In [180]:
#Initialize U and V
d=2
U = np.ones((M.shape[0],d))
V= np.ones((d,M.shape[1]))
#Update U[0,0]
r=0
s=0
U[r,s]=np.sum([V[s,j]*(M_norm[r,j]-np.sum(U[r,:]*V[:,j])+U[r,s]*V[s,j])  for j in range(M.shape[1])])/np.sum(V[0,:]**2)
V[r,s]=np.sum([U[i,r]*(M_norm[i,s]-np.sum(U[i,:]*V[:,s])+U[i,r]*V[r,s])  for i in range(M.shape[0])])/np.sum(U[:,0]**2)

print(M_norm)
print(U[0,0],V[0,0])
print(np.matmul(U,V))

[[ 1.75833333 -1.47        0.09166667  0.59166667 -1.02      ]
 [ 1.15833333 -1.07       -0.50833333  1.99166667 -1.62      ]
 [-0.14166667         nan  0.19166667 -1.30833333  1.08      ]
 [-1.44166667  1.33       -0.10833333 -0.60833333  0.78      ]
 [-1.44166667  1.33       -0.10833333 -0.60833333  0.78      ]
 [ 0.10833333 -0.12        0.44166667 -0.05833333         nan]]
-1.0096666666666667 -1.2499524456380038
[[ 2.26203532 -0.00966667 -0.00966667 -0.00966667 -0.00966667]
 [-0.24995245  2.          2.          2.          2.        ]
 [-0.24995245  2.          2.          2.          2.        ]
 [-0.24995245  2.          2.          2.          2.        ]
 [-0.24995245  2.          2.          2.          2.        ]
 [-0.24995245  2.          2.          2.          2.        ]]


In [183]:
##Now set it up as a loop, running down U and V in order (by whichever dimension first)
d=2
U = np.ones((M.shape[0],d))
V= np.ones((d,M.shape[1]))


for r in range(M.shape[0]):
    for s in range(d):
        U[r,s]=np.nansum([V[s,j]*(M_norm[r,j]-np.sum(U[r,:]*V[:,j])+U[r,s]*V[s,j])  for j in range(M.shape[1])])/np.nansum(V[s,:]**2)
for s in range(M.shape[1]):
    for r in range(d):
        V[r,s]=np.nansum([U[i,r]*(M_norm[i,s]-np.sum(U[i,:]*V[:,s])+U[i,r]*V[r,s])  for i in range(M.shape[0])])/np.nansum(U[:,r]**2)

print(U)
print(V)
    
    

[[-1.00966667  1.        ]
 [-1.00966667  1.        ]
 [-0.83566667  0.63286667]
 [-1.00966667  1.        ]
 [-1.00966667  1.        ]
 [-0.72566667  0.65486667]]
[[0.9521419  0.84482508 0.9809326  0.90493269 0.89680877]
 [0.99723803 0.79821093 0.97813864 1.04854499 0.76608305]]


We've done a step!  Let's see how we're doing.

<br>

### Exercise 3: Back to M

To go back to doing inference in M, we have to do 2 things: compute $P=UV$, then undo our normalization step.  The final result can be compared to M!

Recall that our current scoring metric is RMSE:

$$\sqrt{\frac{1}{n} \sum_{i,j} (M_{i,j} - P_{i,j})^2} $$


In [184]:
P=np.matmul(U,V)

#Put M_sub into P
P_unnorm= P-M_sub
print(P_unnorm)
print(M)
def RMSE(M1, M2):
    rmse=np.sqrt(np.nansum((M1-M2)**2)/(M1.shape[0]*M1.shape[1]-np.sum(np.isnan(M1))))
    return rmse

RMSE(M,P_unnorm)



[[3.27755875 3.41521921 3.89605703 3.54319795 3.88060514]
 [1.87755875 2.01521921 2.49605703 2.14319795 2.48060514]
 [1.97711212 2.16916893 2.607632   2.21570042 2.65539524]
 [3.47755875 3.61521921 4.09605703 3.74319795 4.08060514]
 [3.47755875 3.61521921 4.09605703 3.74319795 4.08060514]
 [3.85378697 4.02966033 4.48705363 4.08831101 4.52089803]]
[[ 5.  2.  4.  4.  3.]
 [ 3.  1.  2.  4.  1.]
 [ 2. nan  3.  1.  4.]
 [ 2.  5.  4.  3.  5.]
 [ 2.  5.  4.  3.  5.]
 [ 4.  4.  5.  4. nan]]


1.0157677537544327

It's hard to say that we're doing great after one iteration, but we could at least check that we've done better than the RMSE from the all-ones initializations.

In [186]:
U = np.ones((M.shape[0],d))
V= np.ones((d,M.shape[1]))

P=np.matmul(U,V)
P_unnorm= P-M_sub

print(RMSE(M,P_unnorm))
print(RMSE(M_norm, P))

2.242624817357681
2.242624817357681


Note: we could (and probably should!) also save a little time by just computing the RMSE of $P$ compared directly to $M_{norm}$.  


**Contemplate**: how does doing the RMSE calculation change depending on which one we use?

<br>

### Exercise 4: Bring Order to the Galaxy

Are we convinced that order really matters?  Repeat exercise 2, but instead of looping in a structured format over the rows and columns, create a random ordering of the $u_{rs}$ indices and another random ordering of the $v_{rs}$ indices, then pass those into the inside of your loop.

In [ ]:
#initialize U and V again
U = np.ones((M.shape[0],d))
V= np.ones((d,M.shape[0]))
indices_list=[(x, y) for x in range(M.shape[0]) for y in range(d)]

#randomize the u's update order
U_index_order = # TODO
#randomize the v's update order
U_index_order = # TODO

for 
    U[u_index_order]= # TODO
    V[v_index_order]= # TODO


Any better?  Any different?

In [ ]:
P=np.matmul(U,V)
#Put M_sub into P
P_unnorm= # TODO
RMSE(M,P_unnorm)